In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import random
import datetime as dt


# Funkcje do obliczania wartości opcji

In [ ]:
# option = {  "option_spec": [,]
#             "position": , 
#             "nominal":,
#             "strike":,
#             "expiry":
# }

def startegy(options, S, nominal_id = False):
    profit = 0
    for element in options:
        option_type, call_put = element["option_spec"]
        if nominal_id:
            position, nominal, strike = element["position"], 1, element["strike"]   
        else:
            position, nominal, strike = element["position"], element["nominal"], element["strike"]
            
        if option_type == "vanilla":
            profit += position * nominal * max(0, call_put * (S - strike))
        elif option_type == "con":
            profit += strike * position * nominal * (call_put * (S - strike) > 0)
        elif option_type == "aon":
            profit += S * position * nominal * (call_put * (S - strike) > 0)
    return profit

def strategy_value(options, S, sigma, df_d, df_f):
    value = 0
    logS = np.log(S)
    for element in options:
        opt_type, sign = element["option_spec"]
        pos, nominal, strike, expiry_date = element["position"], element["nominal"], element["strike"], element["expiry"]

        logK = np.log(element["strike"]) 
        d_1 = (logS - logK + np.log(df_f/df_d) + (sigma**2 / 2) * expiry_date) / (sigma * np.sqrt(expiry_date))
        d_2 = d_1 - sigma * np.sqrt(expiry_date)

        n_1 = stats.norm.cdf(sign * d_1)
        n_2 = stats.norm.cdf(sign * d_2)

        if opt_type == "vanilla": 
            value += pos * nominal * sign * (df_f * S * n_1 - df_d * strike * n_2)
        elif opt_type == "con":   
            value += pos * nominal * sign * (df_d * n_2)
        elif opt_type == "aon":
            value += pos * nominal * sign * (df_f * n_1)
    return value

def unique_strikes(options):
    strikes = [e["strike"] for e in options]
    return np.unique(strikes)

def plot_strategy_value(S_range, options, nominal_id = True, normed = True):
    values = [startegy(options, S, nominal_id) for S in S_range]
    strikes = unique_strikes(options)
    if normed:
        values = values / max(values)

    plt.figure(figsize=(8, 5))
    for strike in strikes:
        plt.axvline(x=strike, color='g', linestyle='--', alpha=0.3)
    plt.plot(S_range, values, linestyle='-', linewidth=2, color='r', label="Strategy Value")
    plt.xlabel("S", fontsize=12, fontweight='bold')
    plt.ylabel("Wypłata ze strategii", fontsize=12, fontweight='bold')
    plt.title("Wypłata ze strategii", fontsize=14, fontweight='bold')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xlim([min(S_range), max(S_range)])
    # plt.show()
    
def s_pdf(x, S, df_d, df_f, sigma, T):
    rd, rf = np.log(df_d)/T, np.log(df_f)/T
    exponent = -0.5 * ((np.log(x/S) - (rd - rf - (sigma**2)/2) * T) / (sigma * np.sqrt(T)))**2
    return (1 / (x * sigma * np.sqrt(2 * np.pi * T))) * np.exp(exponent)

def generate_fx_options(num_options=5, expiry=0.5):
    option_types = ["aon", "con", "vanilla"]
    strikes = [random.uniform(4.5, 4.75) for _ in range(num_options)]
    options = []
    
    for i in range(num_options):
        option = {
            "option_spec": [random.choice(option_types), random.choice([1, -1])],
            "position": random.choice([-1, 1]),
            "nominal": random.uniform(10000, 100000),
            "strike": strikes[i],
            "expiry": expiry
        }
        options.append(option)
    
    return options

def integrand(x, options, S, df_d, df_f, sigma, T):
    return df_d * startegy(options, x) * s_pdf(x, S, df_d, df_f, sigma, T)

def check_for_cross(x):
    if x["instrument"] == "FX Fwd":
        return x["df_f"] * x["nominal_ccy1"] * x["position"]
    else:            
        return x["call_put"] * x["df_f"] * x["nominal_ccy1"] * x["position"] * stats.norm.cdf(x["call_put"] * x["d_1"])

            
def cross_delta(x):
    delta_ccy_1 = x["call_put"] * x["df_f"] * x["nominal_ccy1"] * x["position"] * stats.norm.cdf(x["call_put"] * x["d_1"])
    
    option = {  "option_spec": [x["instrument"], x["call_put"]],
    "position": x["position"], 
    "nominal": x["nominal_ccy2"],
    "strike": x["strike"],
    "expiry": x["expiry"]
    }

    delta_ccy_2 = strategy_value([option], x["fx_spot"], x["sigma"], x["df_d"], x["df_f"]) - x["fx_spot"] * delta_ccy_1

    return [delta_ccy_1, delta_ccy_2]

def calculate_delta(options, currency_data, t_spot):

    cross = ["EUR/USD"]
    options_curr = pd.merge(options, currency_data, on = "fx_rate")
    options_curr["expiry"] = (options_curr["expiry"].apply(pd.to_datetime) - t_spot).dt.days/365

    options_curr["call_put"] = options_curr["call_put"].apply(lambda x: 1 if x == "call" else -1)
    options_curr["position"] = options_curr["position"].apply(lambda x: 1 if x == "buy" else -1)

    options_curr["df_d"] = np.exp(-options_curr["r_ccy2"] * options_curr["expiry"])
    options_curr["df_f"] = np.exp(-options_curr["r_ccy1"] * options_curr["expiry"])

    options_curr["d_1"] = (np.log(options_curr["fx_spot"]/options_curr["strike"]) + np.log(options_curr["df_f"]/options_curr["df_d"])
                            + (options_curr["sigma"]**2 / 2) * options_curr["expiry"]) / (options_curr["sigma"] * np.sqrt(options_curr["expiry"]))
    

    options_curr["delta_cross_ccy_1"] = options_curr.apply(lambda x: check_for_cross(x) if x["fx_rate"] not in cross else cross_delta(x)[0], axis = "columns")
    options_curr["delta_currency"] = options_curr["fx_rate"].apply(lambda x: x[:3])
    options_curr["delta_cross_ccy_2"] = options_curr.apply(lambda x: 0 if x["fx_rate"] not in cross else cross_delta(x)[1], axis = "columns")
    options_curr["delta_currency_2"] = options_curr["fx_rate"].apply(lambda x: x[4:])

    return options_curr[["fx_rate", "instrument", "call_put", "position", "nominal_ccy1", "nominal_ccy2", "expiry", "strike","delta_cross_ccy_1","delta_currency","delta_cross_ccy_2","delta_currency_2"  ]]

    
def portfolio_delta(options, currency_data, t_spot):
     
    data = calculate_delta(options, currency_data, t_spot)

    grouped_1 = data.groupby("delta_currency")[["delta_cross_ccy_1"]].sum().reset_index()
    grouped_2 = data.groupby("delta_currency_2")[["delta_cross_ccy_2"]].sum().reset_index()

    grouped_1.rename(columns={"delta_currency": "currency", "delta_cross_ccy_1": "delta"}, inplace=True)
    grouped_2.rename(columns={"delta_currency_2": "currency", "delta_cross_ccy_2": "delta"}, inplace=True)

    final_grouped = pd.concat([grouped_1, grouped_2]).groupby("currency", as_index=False).sum()
    return final_grouped




In [40]:
t_spot = pd.to_datetime("2025/01/15")

data_por = pd.read_excel("fx_portfolio.xlsx", sheet_name = "Portfolio")
data_market = pd.read_excel("fx_portfolio.xlsx", sheet_name = "Market data")

portfolio_delta(data_por, data_market, t_spot)

,currency,delta
0,EUR,-103546.912648
1,PLN,0.000000
2,USD,44279.036201
